In [1]:
from splinter import Browser
from bs4 import BeautifulSoup as bs
import time
import pandas as pd 
import pymongo
import urllib.request
from urllib.request import urlopen
import requests
import shutil
from pprint import pprint

In [2]:
# Set executable path and initialize chrome browser in splinter
def init_browser():
    executable_path = {"executable_path":'C:\ChromeSafe\chromedriver.exe'}
    return Browser("chrome", **executable_path, headless=False)

In [3]:
def scrape_info():
    browser = init_browser()

In [4]:
scrape_info()

In [5]:
#Create empty dictionaries to store all the mars information.
mars_info_dict=dict()

In [6]:
executable_path = {'executable_path': 'C:\ChromeSafe\chromedriver.exe'}
browser = Browser('chrome', **executable_path, headless=False)

# Part 1 Visit the NASA mars news site

In [7]:
# Visit NASA news Url
url ="https://mars.nasa.gov/news/"
browser.visit(url)

In [8]:
html =browser.html
soup = bs(html, 'html.parser')
slide_elem = soup.select_one('ul.item_list li.slide')

In [10]:
#slide_elem.find("div", class_='content_title').text
slide_elem.find("div", class_='content_title')
#news_title = article.find("div", class_="content_title").text

<div class="content_title"><a href="/news/8432/insight-captures-sunrise-and-sunset-on-mars/" target="_self">InSight Captures Sunrise and Sunset on Mars</a></div>

In [11]:
# use the parent element to find the date in the article
news_date = slide_elem.find("div", class_='list_date').get_text()
news_date

'May  1, 2019'

In [12]:
# use the parent element to find the first a tag and save it as news title
news_title = slide_elem.find("div", class_='content_title').get_text()
news_title

'InSight Captures Sunrise and Sunset on Mars'

In [13]:
# use the parent element to find the paragraph text
news_p = slide_elem.find('div', class_="article_teaser_body").get_text()
news_p

"InSight joins the rest of NASA's Red Planet surface missions, all of which have photographed either the start or end of a Martian day."

In [14]:
#Append results from part 1 into the final mars_info dictionary.
mars_info_dict["Mars_news_date"] = news_date
mars_info_dict["Mars_news_title"] = news_title
mars_info_dict["Mars_news_body"] = news_p
pprint(mars_info_dict)

{'Mars_news_body': "InSight joins the rest of NASA's Red Planet surface "
                   'missions, all of which have photographed either the start '
                   'or end of a Martian day.',
 'Mars_news_date': 'May  1, 2019',
 'Mars_news_title': 'InSight Captures Sunrise and Sunset on Mars'}


# Part 2 JPL Space Images Featured Image

In [20]:
# Visit the JPL Mars URL
url2 = "https://jpl.nasa.gov/spaceimages/?search=&category=Mars"
browser.visit(url2)

In [21]:
html2 = browser.html
soup2 = bs(html2, "html.parser")

In [22]:
# Scrape the browser into soup and use soup to find the image of mars
full_image_elem = browser.find_by_id('full_image')
full_image_elem.click()
# Save the image url to a variable called `img_url`


In [23]:
# find the more info button and click
browser.is_element_present_by_text('more info', wait_time=1)
more_info_elem = browser.find_link_by_partial_text('more info')
more_info_elem.click()

In [24]:
# This will parse the resulting html with soup
html2 = browser.html
soup2 = bs(html2, "html.parser")

In [25]:
# find the relative image url
#img_url_rel = soup2.select_one('figure.lede a img').get("src")
img_url_rel = soup2.find('figure', class_='lede').a['href']
img_url_rel_url = "https://www.jpl.nasa.gov" + img_url_rel
print(img_url_rel_url)

https://www.jpl.nasa.gov/spaceimages/images/largesize/PIA20318_hires.jpg


In [26]:
# Use the base url to create an absolute url
#img_url = f'https://www.jpl.nasa.gov{img_url_rel}'
#img_url

In [27]:
#Append featured image url to the Mars dictionary.
mars_info_dict["Mars_featured_image_url"] = img_url_rel_url

In [28]:
pprint(mars_info_dict)

{'Mars_featured_image_url': 'https://www.jpl.nasa.gov/spaceimages/images/largesize/PIA20318_hires.jpg',
 'Mars_news_body': "InSight joins the rest of NASA's Red Planet surface "
                   'missions, all of which have photographed either the start '
                   'or end of a Martian day.',
 'Mars_news_date': 'May  1, 2019',
 'Mars_news_title': 'InSight Captures Sunrise and Sunset on Mars'}


# 3 Visit the Mars Weather twitter account

In [29]:
url3 = "https://twitter.com/marswxreport?lang=en"
browser.visit(url3)

In [30]:
html3 = browser.html
soup3 = bs(html3, "html.parser")

In [31]:
# find tweet with data name for mars weather
mars_weather = soup3.find('p', class_="TweetTextSize TweetTextSize--normal js-tweet-text tweet-text").text
mars_weather

'InSight sol 152 (2019-05-01) low -98.1ºC (-144.5ºF) high -17.2ºC (1.0ºF)\nwinds from the SW at 4.8 m/s (10.7 mph) gusting to 13.2 m/s (29.5 mph)\npressure at 7.40 hPapic.twitter.com/zJGo11uNF1'

In [32]:
#Add weather tweet to the mars_info dict.
mars_info_dict["Mars_tweet_weather"] = mars_weather

In [33]:
pprint(mars_info_dict)

{'Mars_featured_image_url': 'https://www.jpl.nasa.gov/spaceimages/images/largesize/PIA20318_hires.jpg',
 'Mars_news_body': "InSight joins the rest of NASA's Red Planet surface "
                   'missions, all of which have photographed either the start '
                   'or end of a Martian day.',
 'Mars_news_date': 'May  1, 2019',
 'Mars_news_title': 'InSight Captures Sunrise and Sunset on Mars',
 'Mars_tweet_weather': 'InSight sol 152 (2019-05-01) low -98.1ºC (-144.5ºF) '
                       'high -17.2ºC (1.0ºF)\n'
                       'winds from the SW at 4.8 m/s (10.7 mph) gusting to '
                       '13.2 m/s (29.5 mph)\n'
                       'pressure at 7.40 hPapic.twitter.com/zJGo11uNF1'}


# 4 Mars Facts

In [34]:
df = pd.read_html('https://space-facts.com/mars/')[0]
df.columns=['description', 'value']
df.set_index('description', inplace=True)
df

,value
description,
Equatorial Diameter:,"6,792 km"
Polar Diameter:,"6,752 km"
Mass:,6.42 x 10^23 kg (10.7% Earth)
Moons:,2 (Phobos & Deimos)
Orbit Distance:,"227,943,824 km (1.52 AU)"
Orbit Period:,687 days (1.9 years)
Surface Temperature:,-153 to 20 °C
First Record:,2nd millennium BC
Recorded By:,Egyptian astronomers


In [35]:
df.to_html("mars_facts.html", index=False)

In [36]:
# Provide column names for the dataframe. 
#df.columns = ['Mars_Facts', 'Values']

#Use Pandas to convert the data to a HTML table string.
df.to_html("mars_facts.html", index=False)

#set index for better retrieval. 
#df.set_index("Mars_Facts")
mars_facts_html = df.to_html(classes="description table table-striped")

In [37]:
mars_info_dict["Mars_facts_table"] = mars_facts_html

In [38]:
pprint(mars_info_dict)

{'Mars_facts_table': '<table border="1" class="dataframe description table '
                     'table-striped">\n'
                     '  <thead>\n'
                     '    <tr style="text-align: right;">\n'
                     '      <th></th>\n'
                     '      <th>value</th>\n'
                     '    </tr>\n'
                     '    <tr>\n'
                     '      <th>description</th>\n'
                     '      <th></th>\n'
                     '    </tr>\n'
                     '  </thead>\n'
                     '  <tbody>\n'
                     '    <tr>\n'
                     '      <th>Equatorial Diameter:</th>\n'
                     '      <td>6,792 km</td>\n'
                     '    </tr>\n'
                     '    <tr>\n'
                     '      <th>Polar Diameter:</th>\n'
                     '      <td>6,752 km</td>\n'
                     '    </tr>\n'
                     '    <tr>\n'
                     '      <th>Mass:</th>\n

# 5 Visit the USGS Astogeology site and scrape pictures of the hemispheres

In [59]:
import datetime

In [48]:
url5 =  "https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars/"
browser.visit(url5)

In [49]:
time.sleep(10)
html5 = browser.html
soup5 = bs(html5, "html.parser")

In [50]:
#parse soup object for images of the 4 hemispheres .
class_collap_results = soup5.find('div', class_="collapsible results")
astro_hemis_items = class_collap_results.find_all('div',class_='item')


In [51]:
astro_hemis_items

[<div class="item"><a class="itemLink product-item" href="/search/map/Mars/Viking/cerberus_enhanced"><img alt="Cerberus Hemisphere Enhanced thumbnail" class="thumb" src="/cache/images/dfaf3849e74bf973b59eb50dab52b583_cerberus_enhanced.tif_thumb.png"/></a><div class="description"><a class="itemLink product-item" href="/search/map/Mars/Viking/cerberus_enhanced"><h3>Cerberus Hemisphere Enhanced</h3></a><span class="subtitle" style="float:left">image/tiff 21 MB</span><span class="pubDate" style="float:right"></span><br/><p>Mosaic of the Cerberus hemisphere of Mars projected into point perspective, a view similar to that which one would see from a spacecraft. This mosaic is composed of 104 Viking Orbiter images acquired…</p></div> <!-- end description --></div>,
 <div class="item"><a class="itemLink product-item" href="/search/map/Mars/Viking/schiaparelli_enhanced"><img alt="Schiaparelli Hemisphere Enhanced thumbnail" class="thumb" src="/cache/images/7677c0a006b83871b5a2f66985ab5857_schiapa

In [55]:
# get the list of all the hemispheres
hemis_img_urls_list=list()
img_urls_list = list()
title_list = list()

# Next loop through the links
for i in astro_hemis_items:
    i_title = i.h3.text
    title_list.append(i_title)
    
    # find the href link.
    i_href  = "https://astrogeology.usgs.gov" + i.find('a',class_='itemLink product-item')['href']
    
    print(i_title,i_href)
    
    #browse the link from each page
    browser.visit(i_href)
    time.sleep(5)
    
    #Retrieve the  image links and store in a list. 
    html5   = browser.html
    soup_img = bs(html5, 'html.parser')
    i_img_url = soup_img.find('div', class_='downloads').find('li').a['href']
    print("i_img_url" + i_img_url)
    img_urls_list.append(i_img_url)
    
    # create a dictionary with  each image and title and append to a list. 
    hemispheres_dict = dict()
    hemispheres_dict['title'] = i_title
    hemispheres_dict['img_url'] = i_img_url
    
    hemis_img_urls_list.append(hemispheres_dict)
    
print(hemis_img_urls_list)
print(title_list)
print(img_urls_list)
    
    # finding the elements on each loop
    #browser.find_by_css("a.product-item h3")[i].click()
    
    # Next find the sample  image anchor tag and extract the href
    #sample_elem = browser.find_link_by_text('sample').first
    #asto_hemis['img_url'] = sample_elem['href']
    
    # Get hemisphere title
    #asto_hemis['title'] = browser.find_by_css('h2.title').text
    
    # Append hemisphere object to list
    #hemis_image_urls.append(asto_hemis)
    
    # Navigate back
#browser.back()

Cerberus Hemisphere Enhanced https://astrogeology.usgs.gov/search/map/Mars/Viking/cerberus_enhanced
i_img_urlhttp://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif/full.jpg
Schiaparelli Hemisphere Enhanced https://astrogeology.usgs.gov/search/map/Mars/Viking/schiaparelli_enhanced
i_img_urlhttp://astropedia.astrogeology.usgs.gov/download/Mars/Viking/schiaparelli_enhanced.tif/full.jpg
Syrtis Major Hemisphere Enhanced https://astrogeology.usgs.gov/search/map/Mars/Viking/syrtis_major_enhanced
i_img_urlhttp://astropedia.astrogeology.usgs.gov/download/Mars/Viking/syrtis_major_enhanced.tif/full.jpg
Valles Marineris Hemisphere Enhanced https://astrogeology.usgs.gov/search/map/Mars/Viking/valles_marineris_enhanced
i_img_urlhttp://astropedia.astrogeology.usgs.gov/download/Mars/Viking/valles_marineris_enhanced.tif/full.jpg
[{'title': 'Cerberus Hemisphere Enhanced', 'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif/full.jpg'}

In [56]:
#Add hemispheres list  to the mars_info dictionary.
mars_info_dict["Hemisphere_image_urls"] = hemis_img_urls_list

In [57]:
pprint(mars_info_dict)

{'Hemisphere_image_urls': [{'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif/full.jpg',
                            'title': 'Cerberus Hemisphere Enhanced'},
                           {'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/schiaparelli_enhanced.tif/full.jpg',
                            'title': 'Schiaparelli Hemisphere Enhanced'},
                           {'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/syrtis_major_enhanced.tif/full.jpg',
                            'title': 'Syrtis Major Hemisphere Enhanced'},
                           {'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/valles_marineris_enhanced.tif/full.jpg',
                            'title': 'Valles Marineris Hemisphere Enhanced'}],
 'Mars_facts_table': '<table border="1" class="dataframe description table '
                     'table-striped">\n'
                     '  <thead>\n

In [60]:
#retrieve article date and time and store it in the dictionary.
art_datetime = datetime.datetime.utcnow()
mars_info_dict["Date_time"] = art_datetime

In [61]:
pprint(mars_info_dict)

{'Date_time': datetime.datetime(2019, 5, 3, 17, 14, 20, 345487),
 'Hemisphere_image_urls': [{'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif/full.jpg',
                            'title': 'Cerberus Hemisphere Enhanced'},
                           {'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/schiaparelli_enhanced.tif/full.jpg',
                            'title': 'Schiaparelli Hemisphere Enhanced'},
                           {'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/syrtis_major_enhanced.tif/full.jpg',
                            'title': 'Syrtis Major Hemisphere Enhanced'},
                           {'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/valles_marineris_enhanced.tif/full.jpg',
                            'title': 'Valles Marineris Hemisphere Enhanced'}],
 'Mars_facts_table': '<table border="1" class="dataframe description table '
         